In [1]:
import pandas as pd
import numpy as np
import os
from env import host, user, password
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
import statistics
import seaborn as sns
import env
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from sklearn.impute import SimpleImputer
import scipy
import sklearn.linear_model
import sklearn.preprocessing
import warnings
warnings.filterwarnings("ignore")
import wrangle
import scipy.stats as stats

### 1. Ask at least 5 questions about the data, keeping in mind that your target variable is logerror. e.g. Is logerror significantly different for properties in LA County vs Orange County vs Ventura County?

### 2. Answer those questions through a mix of statistical tests and visualizations.

### 3. Bonus: Compute the mean(logerror) by zipcode and the overall mean(logerror). Write a loop that will run a t-test between the overall mean and the mean for each zip code. We want to identify the zip codes where the error is significantly higher or lower than the expected error.

## Before I get further down any rabbit hole, let's scale and split this data please

The wrange_zillow function is pretty good, just needs to be added to scale and split i think.

# NOTES TO THE EFFECT:
I have moved a number of operations from the wrangle function to the clean_and_prep function--i want the wrangle function to be a simple wrapper for all the other functions in the script
    - I'm expecting errors as a result that will need troubleshooting

In [3]:
# df, train, validate, test, X_train, y_train, X_validate, y_validate, X_test, y_test, \
# train_scaled, X_train_scaled, y_train_scaled, validate_scaled, X_validate_scaled, \
# y_validate_scaled, test_scaled, X_test_scaled, y_test_scaled \
# = wrangle.wrangle_zillow()


In [4]:
df = wrangle.get_zillow_data()
df.shape

(77574, 68)

In [5]:
df = wrangle.clean_and_prep_data(df)
df.shape

(61427, 27)

In [6]:
df.head().T

,1,2,3,4,5
parcelid,17052889,14186244,12177905,10887214,17143294
bathrooms,1.0,2.0,3.0,3.0,2.0
bedrooms,2.0,3.0,4.0,3.0,3.0
condition,6.0,6.0,8.0,8.0,6.0
sq_ft,1465.0,1243.0,2376.0,1312.0,1492.0
county,6111.0,6059.0,6037.0,6037.0,6111.0
full_baths,1.0,2.0,3.0,3.0,2.0
latitude,34449266.0,33886168.0,34245180.0,34185120.0,34230044.0
longitude,-119281531.0,-117823170.0,-118240722.0,-118414640.0,-118993991.0
lot_size,12647.0,8432.0,13038.0,278581.0,903.0


In [8]:
train, validate, test, X_train, y_train, X_validate, y_validate, X_test, y_test = \
wrangle.split_zillow(df)

In [11]:
scaler = sklearn.preprocessing.MinMaxScaler()
scaler

MinMaxScaler()

In [ ]:
train_scaled = scaler.fit_transform(train[num_vars])


# COMPLETE AND RUN YOUR CLEAN_AND_PREP FUNCTION BEFORE PROCEEDING FOR CLARITY
#### adding it inside of 'wrangle', or as a separate step?

In [ ]:
df.dtypes

In [ ]:
for col in df.columns:
    print(df[col].value_counts())

In [ ]:
df.full_baths.value_counts()

In [ ]:
df.has_half_bath.value_counts()

In [ ]:
(df.bathrooms - df.full_baths).value_counts()

There aree 13 homes with 2 half baths

In [ ]:
df[(df.bathrooms - df.full_baths) == 1.0]

## Is there a time period that has a higher or lower log error?
Bin the age group--ten bins? And see results.  Hue on county...if that makes a difference

In [ ]:
sns.scatterplot(x='age', y='logerror', data=df)

In [ ]:
sns.relplot(
    data=df,
    y='logerror',
    x='age',
    col=pd.cut(df.age, bins=[0, 20, 40, 60,80,100,120,200]),
)

Things look a little tighter on either end...the 40-60 and the 60-80 look like they have a wider distribution of logerror

In [ ]:
plt.figure(figsize=(20,10))
sns.boxplot(x='age_bin', y='logerror', data= df)

Still looks like there are more outliers in the middle, but this could be due to more data being available in those age bins

In [ ]:
sns.histplot(data=df, x= df[df.age_bin == pd.Interval(0, 20)].logerror)

In [ ]:
plt.figure(figsize=(20,5))
plt.subplot(1,7,1)
sns.histplot(data=df, x= df[df.age_bin == pd.Interval(0, 20)].logerror)
plt.title("0 to 20")
plt.subplot(1,7,2)
sns.histplot(data=df, x= df[df.age_bin == pd.Interval(20, 40)].logerror)
plt.title("20 to 40")
plt.subplot(1,7,3)
sns.histplot(data=df, x= df[df.age_bin == pd.Interval(40, 60)].logerror)
plt.title("40 to 60")
plt.subplot(1,7,4)
sns.histplot(data=df, x= df[df.age_bin == pd.Interval(60, 80)].logerror)
plt.title("60 to 80")
plt.subplot(1,7,5)
sns.histplot(data=df, x= df[df.age_bin == pd.Interval(80, 100)].logerror)
plt.title("80 to 100")
plt.subplot(1,7,6)
sns.histplot(data=df, x= df[df.age_bin == pd.Interval(100, 120)].logerror)
plt.title("100 to 120")
plt.subplot(1,7,7)
sns.histplot(data=df, x= df[df.age_bin == pd.Interval(120, 200)].logerror)
plt.title("120 and Up")

Looking at logerror by age_bin, they are generally normally distributed

Maybe run an ANOVA test to see about the variances? 

First, Levene

In [ ]:
#H0: Variances are equal
#Ha: Variances are not equal

stats.levene(
    df[df.age_bin == pd.Interval(0, 20)].logerror,
    df[df.age_bin == pd.Interval(20,40)].logerror,
    df[df.age_bin == pd.Interval(40,60)].logerror,
    df[df.age_bin == pd.Interval(60,80)].logerror,
    df[df.age_bin == pd.Interval(80,100)].logerror,
    df[df.age_bin == pd.Interval(100,120)].logerror,
    df[df.age_bin == pd.Interval(120,200)].logerror
)

The results show that the null hypothesis is rejected

In [ ]:
df[df.age_bin == pd.Interval(0, 20)].logerror.var()

In [ ]:
df[df.age_bin == pd.Interval(120,200)].logerror.var()

#### Anova testing of the logerror by age_bin

Set Hypothesis
- 𝐻0 : Population means of logerror (by age_bin) are equal
- 𝐻𝑎 : Population means of logerror (by age_bin) are not all equal

In [ ]:
# This cell runs an ANOVA test on the different 'age_bins'
# We are demonstrating statistically that the age correlates to the logerror
f, p = stats.f_oneway(
    df[df.age_bin == pd.Interval(0, 20)].logerror,
    df[df.age_bin == pd.Interval(20,40)].logerror,
    df[df.age_bin == pd.Interval(40,60)].logerror,
    df[df.age_bin == pd.Interval(60,80)].logerror,
    df[df.age_bin == pd.Interval(80,100)].logerror,
    df[df.age_bin == pd.Interval(100,120)].logerror,
    df[df.age_bin == pd.Interval(120,200)].logerror
)
f, p

In [ ]:
# Setting our alpha and returning a statemtent on the validity of the ANOVA test by comparing alpha to the resulting p-value

alpha = 0.05
if p < alpha:
    print("We reject H_O")
else:
    print("We fail to reject $H_{0}$")

#### The anova test shows I can proceed with the understanding that the average logerror is different depending on the age of the home

In [ ]:
sns.relplot(data=df, y='logerror', x='age', col='county')

This relplot shows age and logerror by count; any dignificant differences between the counties aren't clear

### What about a relationship between tax_value and logerror? (also: are these related targets?)

In [ ]:
sns.scatterplot(x=df.tax_value, y=df.logerror, data=df)

In [ ]:
sns.scatterplot(x=df.tax_value, y=df.logerror, data=df, hue='county', alpha=.5)

In [ ]:
sns.relplot(x=df.tax_value, y=df.logerror, data=df, col='county', alpha=.5)

In [ ]:
df[df.county=='Los_Angeles'].shape

In [ ]:
df[df.county=='Orange'].shape

In [ ]:
df[df.county=='Ventura'].shape

### No clear impressions

In [ ]:
sns.lmplot(x='tax_value', y='logerror', data=df, scatter_kws={'alpha':.2},hue='county')

### Look like perfectly flat trend lines